In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import os 
os.chdir('/home/oldrain123/MMD/')
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
import numpy as np
import torch
from tqdm.auto import tqdm
from sklearn.utils import check_random_state
from sampler.sampler_perturbations import sampler_perturbations
from utils_HD import MatConvert, MMDu, TST_MMD_u

In [5]:
class ModelLatentF(torch.nn.Module):
    """Latent space for both domains."""

    def __init__(self, x_in, H, x_out):
        """Init latent features."""
        super(ModelLatentF, self).__init__()
        self.restored = False

        self.latent = torch.nn.Sequential(
            torch.nn.Linear(x_in, H, bias=True),
            torch.nn.Softplus(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.Softplus(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.Softplus(),
            torch.nn.Linear(H, x_out, bias=True),
        )
    def forward(self, input):
        """Forward the LeNet."""
        fealant = self.latent(input)
        return fealant

In [13]:
# Setup for experiments
dtype = torch.float
device = torch.device("cuda:0")
N_per = 100 # permutation times
alpha = 0.05 # test threshold
scales = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
n = 500
x_in = 1
H = 50 
x_out = 50
d = 1
number_perturbations = 2
learning_rate = 0.005
N_epoch = 1000 # number of training epochs
K = 10 # number of trails
N = 100 # number of test sets
N_f = 100.0 # number of test sets (float)
is_cuda = True
N1 = n 

In [14]:
for scale in scales:
    np.random.seed(1102)
    torch.manual_seed(1102)
    torch.cuda.manual_seed(1102)
    Results = np.zeros([1, K])
    Opt_ep = np.zeros([1, K])
    J_star_u = np.zeros([K, N_epoch])
    ep_OPT = np.zeros([K])
    s_OPT = np.zeros([K])
    s0_OPT = np.zeros([K])
    # Repeat experiments K times (K = 10) and report average test power (rejection rate)
    for kk in tqdm(range(K),desc="Experiment"):
        # Initialize parameters
        if is_cuda:
            model_u = ModelLatentF(x_in, H, x_out).cuda()
        else:
            model_u = ModelLatentF(x_in, H, x_out)
        epsilonOPT = MatConvert(np.random.rand(1) * (10 ** (-10)), device, dtype)
        epsilonOPT.requires_grad = True
        sigmaOPT = MatConvert(np.sqrt(np.random.rand(1) * 0.3), device, dtype)
        sigmaOPT.requires_grad = True
        sigma0OPT = MatConvert(np.sqrt(np.random.rand(1) * 0.002), device, dtype)
        sigma0OPT.requires_grad = True
        # Setup optimizer for training deep kernel
        optimizer_u = torch.optim.Adam(list(model_u.parameters())+[epsilonOPT]+[sigmaOPT]+[sigma0OPT], lr=learning_rate) #
        # Generate Blob-D
        np.random.seed(seed=112 * kk + 1 + n)
        s1,s2 = sampler_perturbations(m=n, n=n, d=d, scale=scale, number_perturbations=number_perturbations, seed=1102)
        # REPLACE above line with
        # s1,s2 = sample_blobs(N1)
        # for validating type-I error (s1 ans s2 are from the same distribution)
        if kk==0:
            s1_o = s1
            s2_o = s2
        S = np.concatenate((s1, s2), axis=0)
        S = MatConvert(S, device, dtype)
        # Train deep kernel to maximize test power
        np.random.seed(seed=1102)
        torch.manual_seed(1102)
        torch.cuda.manual_seed(1102)
        for t in range(N_epoch):
            # Compute epsilon, sigma and sigma_0
            ep = torch.exp(epsilonOPT)/(1+torch.exp(epsilonOPT))
            sigma = sigmaOPT ** 2
            sigma0_u = sigma0OPT ** 2
            # Compute output of the deep network
            modelu_output = model_u(S)
            # Compute J (STAT_u)
            TEMP = MMDu(modelu_output, N1, S, sigma, sigma0_u, ep, complete=True)
            mmd_value_temp = -1 * TEMP[0]
            mmd_std_temp = torch.sqrt(TEMP[1] + 10**(-6))
            # STAT_u = torch.div(mmd_value_temp, mmd_std_temp)
            STAT_u = mmd_value_temp - 1e-3 * mmd_std_temp
            J_star_u[kk, t] = STAT_u.item()
            # Initialize optimizer and Compute gradient
            optimizer_u.zero_grad()
            STAT_u.backward(retain_graph=True)
            # Update weights using gradient descent
            optimizer_u.step()
            # Print MMD, std of MMD and J
            if t % 100 == 0:
                print("mmd_value: ", -1 * mmd_value_temp.item(), "mmd_std: ", mmd_std_temp.item(), "Statistic J: ",
                      -1 * STAT_u.item())
        h_u, threshold_u, mmd_value_u = TST_MMD_u(model_u(S), N_per, N1, S, sigma, sigma0_u, ep, alpha, device,
                                                  dtype, complete=True)
        ep_OPT[kk] = ep.item()
        s_OPT[kk] = sigma.item()
        s0_OPT[kk] = sigma0_u.item()
        print(ep, epsilonOPT)
        # Compute test power of deep kernel based MMD
        H_u = np.zeros(N)
        T_u = np.zeros(N)
        M_u = np.zeros(N)
        np.random.seed(1102)
        count_u = 0
        for k in tqdm(range(N), desc="testing"):
            # Generate Blob-D
            np.random.seed(seed=11 * k + 10 + n)
            s1,s2 = sampler_perturbations(m=n, n=n, d=d, scale=scale, number_perturbations=number_perturbations, seed=11*k+10+n)
            # REPLACE above line with
            # s1,s2 = sample_blobs(N1)
            # for validating type-I error (s1 ans s2 are from the same distribution)
            S = np.concatenate((s1, s2), axis=0)
            S = MatConvert(S, device, dtype)
            # Run two sample test (deep kernel) on generated data
            h_u, threshold_u, mmd_value_u = TST_MMD_u(model_u(S), N_per, N1, S, sigma, sigma0_u, ep, alpha, device, dtype, complete=True)
            # Gather results
            count_u = count_u + h_u
            print("MMD-DK:", count_u, "Threshold:", threshold_u, "MMD_value:", mmd_value_u)
            H_u[k] = h_u
            T_u[k] = threshold_u
            M_u[k] = mmd_value_u
        # Print test power of MMD-D
        print("n =",str(n),"--- Test Power of MMD-D: ", H_u.sum()/N_f)
        Results[0, kk] = H_u.sum() / N_f
        print("n =",str(n),"--- Test Power of MMD-D (K times): ",Results[0])
        print("n =",str(n),"--- Average Test Power of MMD-D: ",Results[0].sum()/(kk+1))
    np.save('./Results_Blob_'+str(n)+'_H1_MMD-D',Results)

Experiment:   0%|          | 0/10 [00:00<?, ?it/s]

mmd_value:  -0.0009053945541381836 mmd_std:  nan Statistic J:  nan
mmd_value:  -0.00011098384857177734 mmd_std:  0.0009538347236891369 Statistic J:  -0.0001100300138480882
mmd_value:  1.9282102584838867e-05 mmd_std:  0.0010058953938563197 Statistic J:  2.0287997978695188e-05
mmd_value:  8.386373519897461e-05 mmd_std:  0.001022318995878379 Statistic J:  8.488605419485299e-05
mmd_value:  0.00012589991092681885 mmd_std:  0.0010324125646425408 Statistic J:  0.00012693232349146138
mmd_value:  0.00020204484462738037 mmd_std:  0.001038245033082545 Statistic J:  0.00020308308966046293
mmd_value:  0.00023843348026275635 mmd_std:  0.0010475867484702197 Statistic J:  0.00023948106701122656
mmd_value:  0.0002833902835845947 mmd_std:  0.0010474045423737423 Statistic J:  0.00028443768812696845
mmd_value:  0.0002827271819114685 mmd_std:  0.0010554837658098507 Statistic J:  0.0002837826656772784
mmd_value:  0.00022795796394348145 mmd_std:  0.0010518025636464298 Statistic J:  0.00022900976650712789
ten

testing:   0%|          | 0/100 [00:00<?, ?it/s]

MMD-DK: 0 Threshold: NaN MMD_value: 0.0005264580249786377
MMD-DK: 0 Threshold: NaN MMD_value: 0.00010929256677627563
MMD-DK: 0 Threshold: NaN MMD_value: 0.0004912987351417542
MMD-DK: 0 Threshold: NaN MMD_value: -0.00024878233671188354
MMD-DK: 0 Threshold: NaN MMD_value: -0.0002903789281845093
MMD-DK: 1 Threshold: 0.0006537437438964844 MMD_value: 0.0007908046245574951
MMD-DK: 1 Threshold: NaN MMD_value: -0.00032644718885421753
MMD-DK: 1 Threshold: NaN MMD_value: -8.558481931686401e-05
MMD-DK: 1 Threshold: NaN MMD_value: -0.0005209892988204956
MMD-DK: 1 Threshold: NaN MMD_value: 5.733221769332886e-05
MMD-DK: 1 Threshold: NaN MMD_value: -0.00029165297746658325
MMD-DK: 1 Threshold: NaN MMD_value: 5.6438148021698e-05
MMD-DK: 1 Threshold: NaN MMD_value: -1.1965632438659668e-05
MMD-DK: 1 Threshold: NaN MMD_value: -0.00032333284616470337
MMD-DK: 1 Threshold: NaN MMD_value: 0.00011061877012252808
MMD-DK: 1 Threshold: NaN MMD_value: 0.0003609880805015564
MMD-DK: 1 Threshold: NaN MMD_value: 1.205

testing:   0%|          | 0/100 [00:00<?, ?it/s]

...

MMD-DK: 0 Threshold: NaN MMD_value: 7.136166095733643e-05
MMD-DK: 1 Threshold: 0.002723783254623413 MMD_value: 0.0034055709838867188
MMD-DK: 1 Threshold: NaN MMD_value: -0.00018621981143951416
MMD-DK: 1 Threshold: NaN MMD_value: 0.0009792745113372803
MMD-DK: 2 Threshold: 0.002620607614517212 MMD_value: 0.006774455308914185
MMD-DK: 2 Threshold: NaN MMD_value: 0.0016997158527374268
MMD-DK: 2 Threshold: NaN MMD_value: 0.0021980106830596924
MMD-DK: 3 Threshold: 0.003223136067390442 MMD_value: 0.0035185515880584717
MMD-DK: 3 Threshold: NaN MMD_value: -0.0016364753246307373
MMD-DK: 3 Threshold: NaN MMD_value: -0.0011979937553405762
MMD-DK: 3 Threshold: NaN MMD_value: 0.0013082027435302734
MMD-DK: 3 Threshold: NaN MMD_value: 0.001263350248336792
MMD-DK: 4 Threshold: 0.0030165910720825195 MMD_value: 0.009487733244895935
MMD-DK: 4 Threshold: NaN MMD_value: -0.002150595188140869
MMD-DK: 4 Threshold: NaN MMD_value: -0.001333281397819519
MMD-DK: 4 Threshold: NaN MMD_value: -0.00012248754501342773


Experiment:   0%|          | 0/10 [00:00<?, ?it/s]

mmd_value:  0.0032497942447662354 mmd_std:  0.002050251401101388 Statistic J:  0.0032518444961673366
mmd_value:  0.014649420976638794 mmd_std:  0.005528236977777096 Statistic J:  0.014654949213616571
mmd_value:  0.014684244990348816 mmd_std:  0.005535637695979095 Statistic J:  0.014689780628044794
mmd_value:  0.014724195003509521 mmd_std:  0.005546916562924544 Statistic J:  0.014729741920072446
mmd_value:  0.014776945114135742 mmd_std:  0.0055718078460107266 Statistic J:  0.014782516921981752
mmd_value:  0.014838427305221558 mmd_std:  0.005587127572647251 Statistic J:  0.014844014432794204
mmd_value:  0.014887571334838867 mmd_std:  0.005585135585396915 Statistic J:  0.014893156470424265
mmd_value:  0.014971360564231873 mmd_std:  0.0056232379349820385 Statistic J:  0.014976983802166855
mmd_value:  0.015056535601615906 mmd_std:  0.005652227428646227 Statistic J:  0.015062187829044551
mmd_value:  0.015134051442146301 mmd_std:  0.005679387079002802 Statistic J:  0.015139730829225304
tensor

testing:   0%|          | 0/100 [00:00<?, ?it/s]

MMD-DK: 1 Threshold: 0.0025362372398376465 MMD_value: 0.00538991391658783
MMD-DK: 2 Threshold: 0.0037435442209243774 MMD_value: 0.005401685833930969
MMD-DK: 3 Threshold: 0.002647966146469116 MMD_value: 0.003223702311515808
MMD-DK: 4 Threshold: 0.0034225285053253174 MMD_value: 0.005698040127754211
MMD-DK: 5 Threshold: 0.0027669966220855713 MMD_value: 0.009466901421546936
MMD-DK: 5 Threshold: NaN MMD_value: 0.0024281740188598633
MMD-DK: 6 Threshold: 0.0038827359676361084 MMD_value: 0.0068826824426651
MMD-DK: 7 Threshold: 0.0026845186948776245 MMD_value: 0.004540368914604187
MMD-DK: 7 Threshold: NaN MMD_value: 0.0007279068231582642
MMD-DK: 7 Threshold: NaN MMD_value: -0.0008956044912338257
MMD-DK: 8 Threshold: 0.0026638805866241455 MMD_value: 0.004201456904411316
MMD-DK: 9 Threshold: 0.002844080328941345 MMD_value: 0.003621920943260193
MMD-DK: 10 Threshold: 0.0030251890420913696 MMD_value: 0.014313682913780212
MMD-DK: 10 Threshold: NaN MMD_value: 0.002940848469734192
MMD-DK: 10 Threshold: